## Before we start - some setup:

- Please visit Sendgrid at: https://sendgrid.com/
- Please set up an account - it's free! (at least, for me, right now).
- Once you've created an account, click on:
  - Settings (left sidebar) >> API Keys >> Create API Key (button on top right)
  - Copy the key to the clipboard, then add a new line to your .env file:
    `SENDGRID_API_KEY=xxxx`
  - And also, within SendGrid, go to:
    - Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"
    - and verify that your own email address is a real email address, so that SendGrid can send emails for you.

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from openai import AsyncOpenAI
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [ ]:
load_dotenv(override=True)
sendgrid_api_key= os.getenv("SENDGRID_API_KEY")

if sendgrid_api_key:
    print("got the sendgrid_api_key")
else:
    print('Didnt got the sendgrid_api_key')

### Additional imports and Gemini support

In [ ]:
# Setup for Gemini API
load_dotenv(override=True)

# Get your Google API key from environment variables
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key:
    print("Please set GOOGLE_API_KEY in your .env file")
else:
    print("Google API key found")

# Set up Gemini client
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.5-flash-preview-05-20", openai_client=gemini_client)

### Instructions to feed the AI model

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \\n",
    "a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \\n",
    "You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \\n",
    "a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \\n",
    "You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \\n",
    "a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \\n",
    "You write concise, to the point cold emails."

### Agent definition

In [ ]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=gemini_model
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=gemini_model
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=gemini_model
)

### Streaming a single agent response

In [ ]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

### Running all agents in parallel

In [ ]:
message = "Write a cold sales email"
# Asyncio -> a library to write concurrent code using the async/await syntax
# asyncio is used to run all the three tasks at the same time 
# Remember: it is not multi-threading or multiprocessing 
# It is using an event loop, which will run each one, 
# and whenever it's pausing for waiting for any input/output then it will let another one run.

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")